In [93]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from pathlib import Path
import numpy as np
import pandas as pd
import json

# 1) utils
Bunch of functions that are usefull for our implementation 

In [ ]:
def parse_smoll_response(response):
  start_tag = "<|im_start|>assistant"
  end_tag = "<|im_end|>"
  start_index = response.find(start_tag)

  start_index += len(start_tag)
    
  end_index = response.find(end_tag, start_index)
  if end_index == -1:
        return ""  
    
  raw_response = response[start_index:end_index]
    
  return raw_response.strip()

def parse_document(text):
    """
    We Split documents in paragraph. 
    In each splitted paragraph, we add the title of the current document.
    """
    chunks = text.split("\n\n")  # Découpe le texte en paragraphes
    title = chunks[0].replace("# Title: ", "")  # Extrait le titre

    formatted_chunks = []  
    for chunk in chunks:  # On parcourt chaque paragraphe
        formatted_chunks.append(f"{title}: {chunk}")  # On ajoute le titre devant

    return {"title": title, "chunks": formatted_chunks}  # On retourne le dictionnaire



In [2]:
smoll360m = "HuggingFaceTB/SmolLM2-360M-Instruct"

In [17]:
path = Path("./corpus")

In [56]:
device = "cpu" # for GPU usage or "cpu" for CPU usage
tokenizer = AutoTokenizer.from_pretrained(smoll360m)
model = AutoModelForCausalLM.from_pretrained(smoll360m).to(device)

In [100]:
message = "who are you ?"

In [101]:
messages = [{"role": "user", "content": message}]
input_text=tokenizer.apply_chat_template(messages, tokenize=False)

In [102]:
inputs = tokenizer.encode(input_text, return_tensors="pt").to(device)
outputs = model.generate(inputs, max_new_tokens=500, temperature=0.2, top_p=0.9, do_sample=True)
print(parse_smoll_response(tokenizer.decode(outputs[0])))

I'm a chatbot designed to assist users with their queries and provide helpful information. I was trained on a vast amount of text data from various sources, including Hugging Face's TensorFlow and PyTorch libraries, which enables me to understand and respond to a wide range of questions.


# 2) Chunking 
Here, we are going to chunk the whole set of informations throughout `parse_class_add_title` function.
As result we have the following output : 
```python
{
    "title": "Notice d’Information",
    "chunks": [
        "Notice d’Information: # Title: Notice d’Information",
        "Notice d’Information: Introduction du document.",
        "Notice d’Information: ## Section 1",
        "Notice d’Information: Détails de la première section.",
        "Notice d’Information: ## Section 2",
        "Notice d’Information: Détails de la deuxième section."
    ]
}

We chose to keep the title behind each chunk to conserve the context.


In [31]:
texts = []
for filename in path.glob("*.md"):
    with open(filename) as f:
        texts.append(f.read())
texts[0]

'# NOTICE D’INFORMATION : Contrat Patrimoine Vie Plus\n\nCette notice présente les principales caractéristiques du contrat Patrimoine Vie Plus, distribué par Capital&Co Assurances.\n\n## 1. Objet du Contrat\n\n- Protéger et valoriser l’épargne du souscripteur via un contrat d’assurance vie multisupport.\n\n## 2. Garanties\n\n1. **Garantie en cas de Décès** :  \n   - Capital versé = valeur atteinte des supports à la date du décès.  \n   - Majoration possible selon l’âge de l’assuré (barème présenté en annexe).\n\n2. **Garantie en cas de Vie** :  \n   - Disponibilité de l’épargne sous forme de rachats partiels ou totaux.\n\n## 3. Frais et Charges\n\n- **Frais de Souscription** : 2,5% des versements.  \n- **Frais de Gestion** : 1,2% annuels pour les unités de compte, 0,7% pour le fonds en euros.\n- **Frais de Transfert/Arbitrage** : 2 opérations gratuites par an, puis 15€ par arbitrage supplémentaire.\n\n## 4. Modalités de Règlement des Prestations\n\n- En cas de décès, le bénéficiaire do

Here, we put every chunk created into one set.

In [32]:
chunks = []  

for txt in texts:  # On parcourt chaque document dans la liste `texts`
    parsed_data = parse_document(txt)  # On applique la fonction pour obtenir un dictionnaire
    chunks.extend(parsed_data["chunks"])  # On ajoute tous les chunks à la liste `chunks`

In [33]:
chunks

['# NOTICE D’INFORMATION : Contrat Patrimoine Vie Plus: # NOTICE D’INFORMATION : Contrat Patrimoine Vie Plus',
 '# NOTICE D’INFORMATION : Contrat Patrimoine Vie Plus: Cette notice présente les principales caractéristiques du contrat Patrimoine Vie Plus, distribué par Capital&Co Assurances.',
 '# NOTICE D’INFORMATION : Contrat Patrimoine Vie Plus: ## 1. Objet du Contrat',
 '# NOTICE D’INFORMATION : Contrat Patrimoine Vie Plus: - Protéger et valoriser l’épargne du souscripteur via un contrat d’assurance vie multisupport.',
 '# NOTICE D’INFORMATION : Contrat Patrimoine Vie Plus: ## 2. Garanties',
 '# NOTICE D’INFORMATION : Contrat Patrimoine Vie Plus: 1. **Garantie en cas de Décès** :  \n   - Capital versé = valeur atteinte des supports à la date du décès.  \n   - Majoration possible selon l’âge de l’assuré (barème présenté en annexe).',
 '# NOTICE D’INFORMATION : Contrat Patrimoine Vie Plus: 2. **Garantie en cas de Vie** :  \n   - Disponibilité de l’épargne sous forme de rachats partiels

# Embeddings

In [60]:
from FlagEmbedding import FlagModel

tester différents embeddings

In [61]:
embedder = FlagModel(
    'BAAI/bge-base-en-v1.5',
    query_instruction_for_retrieval="Represent this sentence for searching relevant passages:",
    use_fp16=True,
)

In [62]:
corpus_embedding = embedder.encode(chunks)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [63]:
queries = [
    "Quel est l'objet du contrat patrimoine vie plus ?",
    "Qui est l'assureur du contrat sécurité avenir ?",
]

In [64]:
query_embedding = embedder.encode(queries)

Sim scores is simply a search for similarity between the queries embedding and the corpus embedding.

In [65]:
sim_scores = query_embedding @ corpus_embedding.T
sim_scores

array([[0.825 , 0.812 , 0.855 , 0.8403, 0.8286, 0.8203, 0.8296, 0.832 ,
        0.8203, 0.818 , 0.819 , 0.8315, 0.8315, 0.765 , 0.822 , 0.807 ,
        0.7783, 0.801 , 0.8223, 0.822 , 0.783 , 0.8203, 0.6514, 0.716 ,
        0.7373, 0.755 , 0.709 , 0.7686, 0.689 , 0.7505, 0.7407, 0.749 ,
        0.6377, 0.7764, 0.661 , 0.774 , 0.708 , 0.708 , 0.7085, 0.7617,
        0.7104, 0.744 , 0.692 , 0.7363, 0.748 , 0.7544, 0.7183, 0.7007,
        0.746 , 0.748 , 0.7573, 0.73  , 0.7485, 0.7153, 0.758 , 0.7495,
        0.7705, 0.6973, 0.75  , 0.7524, 0.7666, 0.726 , 0.7705, 0.771 ,
        0.7827, 0.73  , 0.731 , 0.7153, 0.71  , 0.704 , 0.712 , 0.7188,
        0.701 , 0.7163, 0.715 , 0.7334, 0.6987, 0.743 , 0.6787, 0.7266,
        0.699 , 0.7407, 0.727 , 0.7285, 0.7134, 0.708 , 0.7007, 0.7334,
        0.684 , 0.715 , 0.7163, 0.7627, 0.701 , 0.7583, 0.7705, 0.732 ,
        0.743 , 0.724 , 0.716 , 0.71  , 0.716 , 0.712 , 0.742 , 0.7188,
        0.7524, 0.729 , 0.7456, 0.707 , 0.7666, 0.695 , 0.703 , 

In [66]:
for query, score in zip(queries, sim_scores):
    print(" ---- ")
    print("Query: ", query)
    indexes = np.argsort(score)[-5:]
    print("Sources:")
    for i, idx in enumerate(reversed(indexes)):
        if score[idx] > .5:
            print(f"{i+1} -- similarity {score[idx]:.2f} -- \"", chunks[idx], '"')

 ---- 
Query:  Quel est l'objet du contrat patrimoine vie plus ?
Sources:
1 -- similarity 0.85 -- " # NOTICE D’INFORMATION : Contrat Patrimoine Vie Plus: ## 1. Objet du Contrat "
2 -- similarity 0.84 -- " # NOTICE D’INFORMATION : Contrat Patrimoine Vie Plus: - Protéger et valoriser l’épargne du souscripteur via un contrat d’assurance vie multisupport. "
3 -- similarity 0.83 -- " # NOTICE D’INFORMATION : Contrat Patrimoine Vie Plus: ## 3. Frais et Charges "
4 -- similarity 0.83 -- " # NOTICE D’INFORMATION : Contrat Patrimoine Vie Plus: ## 5. Clause Bénéficiaire "
5 -- similarity 0.83 -- " # NOTICE D’INFORMATION : Contrat Patrimoine Vie Plus: - Par défaut : « Le conjoint survivant, à défaut les enfants nés ou à naître, à défaut les héritiers légaux. »
- Le souscripteur peut désigner tout autre bénéficiaire par écrit.   "
 ---- 
Query:  Qui est l'assureur du contrat sécurité avenir ?
Sources:
1 -- similarity 0.86 -- " # CONTRAT D’ASSURANCE VIE : Sécurité Avenir: ## Article 2 : Personnes C

In [67]:
df = pd.read_csv("questions.csv")

In [69]:
query_embedding = embedder.encode(list(df["question"]))
query_embedding

array([[-0.0138  , -0.002901,  0.04266 , ..., -0.05826 ,  0.03516 ,
         0.001739],
       [-0.006912,  0.01024 ,  0.01994 , ..., -0.03494 ,  0.008   ,
         0.03162 ]], dtype=float16)

In [70]:
def compute_mrr(sim_score, acceptable_chunks):
    ranks = []
    for this_score, this_acceptable_chunks in zip(sim_score, acceptable_chunks):
        indexes = reversed(np.argsort(this_score))
        rank = 1 + next(i for i, idx in enumerate(indexes) if idx in this_acceptable_chunks)
        ranks.append(rank)
        
    return {
        "score": sum(1 / r if r < 6 else 0 for r in ranks) / len(ranks),
        "ranks": ranks,
    }

In [104]:
def get_context(query, corpus, corpus_embedding):
    query_embedding = embedder.encode([query])
    sim_scores = query_embedding @ corpus_embedding.T
    indexes = list(np.argsort(sim_scores[0]))[-5:]
    return [corpus[i] for i in indexes]

In [75]:
sim_scores = query_embedding @ corpus_embedding.T
sim_scores

array([[0.825 , 0.812 , 0.855 , 0.8403, 0.8286, 0.8203, 0.8296, 0.832 ,
        0.8203, 0.818 , 0.819 , 0.8315, 0.8315, 0.765 , 0.822 , 0.807 ,
        0.7783, 0.801 , 0.8223, 0.822 , 0.783 , 0.8203, 0.6514, 0.716 ,
        0.7373, 0.755 , 0.709 , 0.7686, 0.689 , 0.7505, 0.7407, 0.749 ,
        0.6377, 0.7764, 0.661 , 0.774 , 0.708 , 0.708 , 0.7085, 0.7617,
        0.7104, 0.744 , 0.692 , 0.7363, 0.748 , 0.7544, 0.7183, 0.7007,
        0.746 , 0.748 , 0.7573, 0.73  , 0.7485, 0.7153, 0.758 , 0.7495,
        0.7705, 0.6973, 0.75  , 0.7524, 0.7666, 0.726 , 0.7705, 0.771 ,
        0.7827, 0.73  , 0.731 , 0.7153, 0.71  , 0.704 , 0.712 , 0.7188,
        0.701 , 0.7163, 0.715 , 0.7334, 0.6987, 0.743 , 0.6787, 0.7266,
        0.699 , 0.7407, 0.727 , 0.7285, 0.7134, 0.708 , 0.7007, 0.7334,
        0.684 , 0.715 , 0.7163, 0.7627, 0.701 , 0.7583, 0.7705, 0.732 ,
        0.743 , 0.724 , 0.716 , 0.71  , 0.716 , 0.712 , 0.742 , 0.7188,
        0.7524, 0.729 , 0.7456, 0.707 , 0.7666, 0.695 , 0.703 , 

In [115]:
get_context("Donne moi les informations complémentaires sur le contrat Patrimoine Vie Plus.", chunks, corpus_embedding)

['# NOTICE D’INFORMATION : Contrat Patrimoine Vie Plus: ## 2. Garanties',
 '# NOTICE D’INFORMATION : Contrat Patrimoine Vie Plus: ## 4. Modalités de Règlement des Prestations',
 '# NOTICE D’INFORMATION : Contrat Patrimoine Vie Plus: - Protéger et valoriser l’épargne du souscripteur via un contrat d’assurance vie multisupport.',
 '# NOTICE D’INFORMATION : Contrat Patrimoine Vie Plus: ## 7. Garanties Complémentaires (Optionnelles)',
 '# NOTICE D’INFORMATION : Contrat Patrimoine Vie Plus: ## 8. Informations Complémentaires']

In [114]:
def build_smoll_messages(query, chunks, corpus_embedding):
    context_str = "\n\n".join(get_context(query, chunks, corpus_embedding))

    messages = [
        {"role": "system", "content": f"""You reply to the user's request using only context information.
        Context information to answer "{query}" is below
        ------
        Context:
        {context_str}
        ------
        You are a helpful assistant for a insurance company. You reply to workers 'questions about the document that they send you.
        """},
        {"role": "user", "content": query},
    ]

    return messages

In [118]:
messages = build_smoll_messages("Donne moi des informations importante sur le contrat patrimoine vie plus", chunks, corpus_embedding)
messages

[{'role': 'system',
  'content': 'You reply to the user\'s request using only context information.\n        Context information to answer "Donne moi des informations importante sur le contrat patrimoine vie plus" is below\n        ------\n        Context:\n        # NOTICE D’INFORMATION : Contrat Patrimoine Vie Plus: - Protéger et valoriser l’épargne du souscripteur via un contrat d’assurance vie multisupport.\n\n# NOTICE D’INFORMATION : Contrat Patrimoine Vie Plus: ## 4. Modalités de Règlement des Prestations\n\n# NOTICE D’INFORMATION : Contrat Patrimoine Vie Plus: ## 1. Objet du Contrat\n\n# NOTICE D’INFORMATION : Contrat Patrimoine Vie Plus: ## 8. Informations Complémentaires\n\n# NOTICE D’INFORMATION : Contrat Patrimoine Vie Plus: ## 2. Garanties\n        ------\n        You are a helpful assistant for a insurance company. You reply to workers \'questions about the document that they send you.\n        '},
 {'role': 'user',
  'content': 'Donne moi des informations importante sur le

In [120]:
messages = build_smoll_messages("Donne moi les informations complémentaires sur le contrat Patrimoine Vie Plus.", chunks, corpus_embedding)

input_text=tokenizer.apply_chat_template(messages, tokenize=False)
inputs = tokenizer.encode(input_text, return_tensors="pt").to(device)
outputs = model.generate(inputs, max_new_tokens=500, temperature=0.1, top_p=0.9, do_sample=True)
response = tokenizer.decode(outputs[0])
print(parse_smoll_response(response))

In [87]:
questions = pd.read_csv('questions.csv')
questions_list = questions['question'].tolist()

In [89]:
embeddings = embedder.encode(questions_list).tolist()  

In [90]:
response_list = []
for question in questions_list:
    messages = build_smoll_messages(question, chunks, corpus_embedding)
    input_text = tokenizer.apply_chat_template(messages, tokenize=False)
    inputs = tokenizer.encode(input_text, return_tensors="pt").to(device)
    outputs = model.generate(
        inputs, 
        max_new_tokens=500, 
        temperature=0.01, 
        top_p=0.9, 
        do_sample=True
    )
    response = tokenizer.decode(outputs[0])
    response_list.append(response)

In [98]:
def create_rag_csv(filename, query_list, answers):
    """
    créer un fichier qui contient la query, les embeddings et les réponses (non-parsées)
    """
    if not (len(query_list) == len(answers)):
        raise ValueError("Les listes questions, embeddings et answers doivent avoir la même longueur.")

    # Préparer les données pour le CSV
    data = [
        {
            "question": question,
            "rag_reply": parse_smoll_response(answer),
        }
        for question, answer in zip(query_list, answers)
    ]

    # Créer un DataFrame et l'enregistrer en CSV
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False)
    print(f"CSV créé : {filename}")

In [99]:
create_rag_csv('SmolLV2_360m.csv', query_list=questions_list, answers=response_list)

CSV créé : SmolLV2_360m.csv
